<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/11_28.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 定義函數映射，擴展到 15 個函數
function_mapping = {
    0: rule_ooc,
    1: rule_trend,
    2: rule_cons_n_out,
    # 添加其他函數的占位，直到15個
    3: lambda df: df.assign(custom_rule_3="Applied custom_rule_3"),
    4: lambda df: df.assign(custom_rule_4="Applied custom_rule_4"),
    5: lambda df: df.assign(custom_rule_5="Applied custom_rule_5"),
    # ...
    14: lambda df: df.assign(custom_rule_14="Applied custom_rule_14"),
}

# 將 tt 的 rule 解碼為長度15的布林列表
tt_df['rule'] = tt_df['rule'].apply(lambda x: list(map(int, str(x).zfill(15))))
# str(x).zfill(15)：保證 rule 長度為 15。
# list(map(int, ...))：將字元轉為整數列表



# 分組處理邏輯
result = pd.DataFrame()
for name, group in data.groupby('name'):
    # 獲取該 name 對應的 rule
    rule = tt_df.loc[tt_df['name'] == name, 'rule'].values
    if len(rule) > 0:
        rule = rule[0]  # 獲取該 name 的第一條規則
        # 按照 rule 應用函數
        for idx, flag in enumerate(rule):
            if flag == 1 and idx in function_mapping:
                func = function_mapping[idx]
                group = func(group)
    result = pd.concat([result, group])

result.reset_index(drop=True, inplace=True)
print(result)


如果每個 rule 函數的參數數量與輸入值不同，解決方式是對每個函數建立對應的參數配置，然後在執行函數時動態傳遞這些參數。以下是實現方式的詳細步驟：

In [ ]:
# 定義函數
def rule_ooc(df, k=3):
    df['rule_ooc'] = f"Applied rule_ooc with k={k}"
    return df

def rule_trend(df, n=7, factor=1.5):
    df['rule_trend'] = f"Applied rule_trend with n={n}, factor={factor}"
    return df

def rule_cons_n_out(df, n=8, threshold=10):
    df['rule_cons_n_out'] = f"Applied rule_cons_n_out with n={n}, threshold={threshold}"
    return df

# 函數映射與參數配置表
function_mapping = {
    0: rule_ooc,
    1: rule_trend,
    2: rule_cons_n_out,
}

# 每個函數對應的參數
function_params = {
    0: {"k": 5},
    1: {"n": 10, "factor": 2.0},
    2: {"n": 12, "threshold": 15},
}

# 模擬資料
data = pd.DataFrame({
    'name': ['A', 'A', 'B', 'C', 'D', 'E', 'A', 'C'],
    'value': [10, 20, 30, 40, 50, 60, 70, 80]
})

# 更新 tt 表，確保 rule 長度為15
tt_df['rule'] = tt_df['rule'].apply(lambda x: list(map(int, str(x).zfill(15))))

# 分組處理邏輯
result = pd.DataFrame()
for name, group in data.groupby('name'):
    # 獲取該 name 對應的 rule
    rule = tt_df.loc[tt_df['name'] == name, 'rule'].values
    if len(rule) > 0:
        rule = rule[0]  # 獲取該 name 的第一條規則
        # 按照 rule 應用函數
        for idx, flag in enumerate(rule):
            if flag == 1 and idx in function_mapping:
                func = function_mapping[idx]
                params = function_params.get(idx, {})  # 獲取對應參數
                group = func(group, **params)  # 傳遞參數執行函數
    result = pd.concat([result, group])

result.reset_index(drop=True, inplace=True)
print(result)


* 明天看

In [ ]:
import ast

# 假設 tt_df 中有一列 'params' 記錄數字參數，例如 '[1,2][2,2][]'
tt_df['params'] = ['[1,2][2,2][]', '[3,4][5,6][7,8]', '[][][9,10]', '[1,1][][2,2]', '[3,3][4,4][]']

# 定義函數
def rule_ooc(df, k=3):
    df['rule_ooc'] = f"Applied rule_ooc with k={k}"
    return df

def rule_trend(df, n=7, factor=1.5):
    df['rule_trend'] = f"Applied rule_trend with n={n}, factor={factor}"
    return df

def rule_cons_n_out(df, n=8, threshold=10):
    df['rule_cons_n_out'] = f"Applied rule_cons_n_out with n={n}, threshold={threshold}"
    return df

# 函數映射
function_mapping = {
    0: rule_ooc,
    1: rule_trend,
    2: rule_cons_n_out,
}

# 字串參數（如果有的話）
function_params = {
    1: {"factor": 2.0},
    2: {"threshold": 15},
}

# 解析 'params' 為數字參數列表
def parse_params(param_str):
    # 將 '[1,2][2,2][]' 解析為 [[1, 2], [2, 2], []]
    return [ast.literal_eval(p) if p else [] for p in param_str.strip('][').split('][')]

tt_df['parsed_params'] = tt_df['params'].apply(parse_params)

# 模擬數據
data = pd.DataFrame({
    'name': ['A', 'A', 'B', 'C', 'D', 'E', 'A', 'C'],
    'value': [10, 20, 30, 40, 50, 60, 70, 80]
})

# 分組處理邏輯
result = pd.DataFrame()
for name, group in data.groupby('name'):
    # 獲取該 name 對應的 rule 和解析參數
    match = tt_df.loc[tt_df['name'] == name]
    if not match.empty:
        rule = match['rule'].values[0]
        params = match['parsed_params'].values[0]
        # 按照 rule 應用函數
        for idx, flag in enumerate(rule):
            if flag == 1 and idx in function_mapping:
                func = function_mapping[idx]
                # 數字參數與字串參數合併
                param_dict = function_params.get(idx, {})
                if idx < len(params) and params[idx]:  # 若數字參數存在
                    param_dict.update({'k': params[idx][0], 'n': params[idx][1]})
                # 執行函數
                group = func(group, **param_dict)
    result = pd.concat([result, group])

result.reset_index(drop=True, inplace=True)
print(result)


1. 數字參數解析 (parse_params)：

將 Excel 中的 [1,2][2,2][] 格式解析為 [[1, 2], [2, 2], []]。
每個內部列表對應於函數的數字參數。
2. 參數合併與動態調用：

根據 rule 和 parsed_params 動態生成函數參數。
將字串參數（如 factor 或 threshold）與解析出的數字參數合併，然後傳遞給函數。
3. 保留靈活性：

支援部分函數無參數（空 []）。
支援字串與數字參數的混合使用。